In [1]:
from sklearn.metrics import adjusted_rand_score as ARI
from sklearn.metrics import adjusted_mutual_info_score as AMI 
import hypernetx as hnx
import hypernetx.algorithms.hypergraph_modularity as hmod
from collections import Counter
import igraph as ig
import pandas as pd
import numpy as np
import h_louvain as hl
import csv
import bayes_opt



In [2]:
def load_ABCDH_from_file(filename):
    with open(filename,"r") as f:
        rd = csv.reader(f)
        lines = list(rd)
    Edges = []
    for line in lines:
        Edges.append(list(line))

    HG = hnx.Hypergraph(dict(enumerate(Edges)))
    return HG

In [3]:
HG = load_ABCDH_from_file("datasets/results_300_he.txt")
## julia --project abcdh.jl -n 300 -d 2.5,5,20 -c 1.5,10,30 -x 0.3 -q 0.0,0.4,0.3,0.2,0.1 -w :linear -s 1234 --stats -o results_300


In [4]:
%%time
# hmod_tau: w(d,c) = (c/d)^tau for c>d/2 else 0
# hmod_tau = 1 (linear) 
# hmod_tau = 0 (majority)
# hmod_tau = "infinity" (strict)

hL = hl.hLouvain(HG, hmod_tau=1, resolution = 1, random_seed = 123) 


CPU times: user 125 ms, sys: 4.37 ms, total: 129 ms
Wall time: 128 ms


In [5]:
## basic hLouvain algorithm execution (without the last_step optimization)

In [6]:
%%time

alphas = []
c = 0.3
b = 0.8
for i in range(30):
    alphas.append(1-((1-b)**i))

#alphas = [0]
#alphas = [1]

A, q2, alphas_out = hL.h_louvain_community(alphas = alphas, change_frequency = c)

print("alphas_out", alphas_out)
print("final_alpha", alphas_out[-1])
print("q2 =",q2)

alphas_out [0.0, 0.8, 0.96, 1]
final_alpha 1
q2 = 0.5309079514468399
CPU times: user 811 ms, sys: 4.17 ms, total: 815 ms
Wall time: 814 ms


In [7]:
#checking the results (calculate h-modularity (alpha =1)) and print partition
qH = hL.combined_modularity(A, alpha = 1,hmod_tau=1,resolution=1)
print("qH", qH)
print("partition",A)

qH 0.5309079514468399
partition [{'109', '88', '247', '46', '12', '85', '143', '18', '210', '116', '288', '237', '173', '97', '72', '128', '155', '188', '223', '228', '152', '151', '222', '20', '106', '251', '229', '193', '78', '265', '249', '95', '145', '132'}, {'283', '62', '235', '290', '13', '147', '186', '177', '180', '114', '4', '82', '270', '44', '40', '32', '167', '63', '236', '162', '255'}, {'142', '294', '36', '163', '298', '170', '131', '99', '42', '254', '69', '59', '264', '144', '47', '10'}, {'207', '187', '3', '250', '129', '33', '165', '221', '176', '127'}, {'226', '280', '198', '28', '71', '68', '92', '41', '123', '189', '110', '209', '141', '83', '121', '154', '185', '268', '16', '90', '87', '119', '34', '89', '230'}, {'67', '104', '224', '73', '35', '45', '201', '216', '233', '166', '205', '202', '258', '48', '140', '241', '15', '225', '200', '102', '244', '269', '49', '31', '234', '211', '220', '172', '2', '266', '58', '29'}, {'212', '100', '118', '246', '240', '287'

In [8]:
## enhanced algorithm execution (with the last step optimization)

In [9]:
%%time

alphas = []
c = 0.3
b = 0.8
for i in range(30):
    alphas.append(1-((1-b)**i))
    
    
Als, A, qH_ls, qH_basic, alphas_out = hL.h_louvain_community_plus_last_step(alphas = alphas, change_frequency = c)


print("alphas_out", alphas_out)
print("final_alpha", alphas_out[-1])
print("qH-basic =",qH_basic)
print("qH+last_step =",qH_ls)


alphas_out [0.0, 0.8, 0.96, 1]
final_alpha 1
qH-basic = 0.5309079514468399
qH+last_step = 0.532611196913002
CPU times: user 3.68 s, sys: 25.8 ms, total: 3.71 s
Wall time: 3.69 s


In [10]:
#checking the results (calculate h-modularity (alpha =1)) and print partition
qH = hL.combined_modularity(A, alpha = 1,hmod_tau=1,resolution=1)
print("qH", qH)
print("partition",A)

qHls = hL.combined_modularity(Als, alpha = 1,hmod_tau=1,resolution=1)
print("qH_last_step", qHls)
print("partition",Als)

qH 0.5309079514468399
partition [{'109', '88', '247', '46', '12', '85', '143', '18', '210', '116', '288', '237', '173', '97', '72', '128', '155', '188', '223', '228', '152', '151', '222', '20', '106', '251', '229', '193', '78', '265', '249', '95', '145', '132'}, {'283', '62', '235', '290', '13', '147', '186', '177', '180', '114', '4', '82', '270', '44', '40', '32', '167', '63', '236', '162', '255'}, {'142', '294', '36', '163', '298', '170', '131', '99', '42', '254', '69', '59', '264', '144', '47', '10'}, {'207', '187', '3', '250', '129', '33', '165', '221', '176', '127'}, {'226', '280', '198', '28', '71', '68', '92', '41', '123', '189', '110', '209', '141', '83', '121', '154', '185', '268', '16', '90', '87', '119', '34', '89', '230'}, {'67', '104', '224', '73', '35', '45', '201', '216', '233', '166', '205', '202', '258', '48', '140', '241', '15', '225', '200', '102', '244', '269', '49', '31', '234', '211', '220', '172', '2', '266', '58', '29'}, {'212', '100', '118', '246', '240', '287'

In [11]:
## ground truth

with open("datasets/results_300_assign.txt", 'r') as file:
    gt = [int(line) for line in file]
A_gt = [x for x in hmod.dict2part({str(i+1):gt[i] for i in range(len(gt))}) if len(x)>0]
gt_mod = hL.combined_modularity(A_gt, alpha = 1,hmod_tau=1,resolution=1)

print("ground-truth partition")
print(A_gt)
print("qh-gt:",gt_mod)

def getAMI_ARI(HG,gt,A):
    d = hmod.part2dict(A)
    A4ari = [d[str(i+1)] for i in range(len(HG.nodes))]
    return ARI(gt, A4ari), AMI(gt, A4ari)

ground-truth partition
[{'81', '260', '285', '14', '148', '135', '124', '65', '272', '271', '286', '125', '117', '70', '112', '197', '203', '79', '300', '191', '134', '254', '182', '144', '77', '227', '183', '96'}, {'226', '198', '280', '28', '71', '68', '92', '41', '123', '189', '110', '209', '141', '83', '121', '154', '185', '268', '16', '90', '87', '119', '34', '89', '230'}, {'109', '88', '296', '247', '46', '12', '221', '143', '18', '210', '116', '288', '173', '72', '223', '152', '228', '151', '222', '251', '229', '78', '265', '95', '266'}, {'283', '62', '235', '290', '13', '147', '186', '177', '261', '180', '114', '4', '82', '270', '44', '40', '32', '167', '63', '236', '162', '255'}, {'67', '104', '195', '35', '45', '201', '174', '216', '166', '258', '140', '200', '102', '269', '49', '31', '299', '257', '172', '2', '58', '29'}, {'103', '179', '80', '190', '213', '93', '169', '284', '238', '241', '66', '57', '149', '50', '211', '297', '43', '276', '74'}, {'214', '279', '178', '138'

In [12]:
#AMI and ARI for basic A

ari, ami = getAMI_ARI(HG,gt,A)
print("ARI =", ari)
print("AMI =", ami)
print("comm =", len(A))
print("comm-gt =", len(A_gt))

ARI = 0.7607284749397808
AMI = 0.8582215418720631
comm = 15
comm-gt = 18


In [13]:
#AMI and ARI for the result after Last Step optimization

ari, ami = getAMI_ARI(HG,gt,Als)
print("ARI =", ari)
print("AMI =", ami)
print("comm =", len(Als))
print("comm-gt =", len(A_gt))

ARI = 0.7502721326846687
AMI = 0.8495035736220896
comm = 15
comm-gt = 18


### Bigger example 

In [14]:
HG = load_ABCDH_from_file("datasets/results_3000_he.txt")
## julia --project abcdh.jl -n 3000 -d 2.5,5,20 -c 1.5,10,30 -x 0.3 -q 0.0,0.4,0.3,0.2,0.1 -w :linear -s 1234 --stats -o results_3000


In [15]:
%%time

hL = hl.hLouvain(HG, hmod_tau=1, resolution = 1, random_seed = 5673) 

CPU times: user 6.74 s, sys: 1.29 ms, total: 6.74 s
Wall time: 6.74 s


In [16]:
%%time

alphas = []
c = 0.3
b = 0.8
for i in range(30):
    alphas.append(1-((1-b)**i))

#alphas = [0]
#alphas = [1]

A, q2, alphas_out = hL.h_louvain_community(alphas = alphas, change_frequency = c)

print("alphas_out", alphas_out)
print("final_alpha", alphas_out[-1])
print("q2 =",q2)

alphas_out [0.0, 0.8, 0.96, 0.992, 1]
final_alpha 1
q2 = 0.568983501700291
CPU times: user 15.6 s, sys: 29 ms, total: 15.7 s
Wall time: 15.6 s


In [17]:
%%time

alphas = []
c = 0.3
b = 0.8
for i in range(30):
    alphas.append(1-((1-b)**i))
    
    
Als, A, qH_ls, qH_basic, alphas_out = hL.h_louvain_community_plus_last_step(alphas = alphas, change_frequency = c)


print("alphas_out", alphas_out)
print("final_alpha", alphas_out[-1])
print("qH-basic =",qH_basic)
print("qH+last_step =",qH_ls)

alphas_out [0.0, 0.8, 0.96, 0.992, 1]
final_alpha 1
qH-basic = 0.568983501700291
qH+last_step = 0.5721205900873205
CPU times: user 57.1 s, sys: 217 ms, total: 57.3 s
Wall time: 57 s


## Weighted graph

In [18]:
import pickle

def load_GoT():
    ## load the GoT dataset
    Edges, Names, Weights = pickle.load(open( "datasets/GoT.pkl", "rb" ))
    print(len(Names),'nodes and',len(Edges),'edges')

    HG = hnx.Hypergraph(dict(enumerate(Edges)))
    ## add edge weights
    for e in HG.edges:
        #HG.dataframe["weight"] = Weights
        HG.edges[e].weight = Weights[e]
    return HG

HG = load_GoT()

198 nodes and 1492 edges


In [19]:
hL = hl.hLouvain(HG, hmod_tau="infinity", resolution = 1, random_seed = 5673) 

In [20]:
%%time

alphas = []
c = 0.3
b = 0.6
for i in range(30):
    alphas.append(1-((1-b)**i))

#alphas = [0]
#alphas = [1]

A, q2, alphas_out = hL.h_louvain_community(alphas = alphas, change_frequency = c)

print("alphas_out", alphas_out)
print("final_alpha", alphas_out[-1])
print("q2 =",q2)

alphas_out [0.0, 0.6, 0.84, 0.9359999999999999, 1]
final_alpha 1
q2 = 0.5700210291240112
CPU times: user 1min 6s, sys: 200 ms, total: 1min 6s
Wall time: 1min 5s


In [21]:
%%time

### second run with different c and b (faster since the degree taxes are precalculated)

alphas = []
c = 0.2
b = 0.7
for i in range(30):
    alphas.append(1-((1-b)**i))

#alphas = [0]
#alphas = [1]

A, q2, alphas_out = hL.h_louvain_community(alphas = alphas, change_frequency = c)

print("alphas_out", alphas_out)
print("final_alpha", alphas_out[-1])
print("q2 =",q2)

alphas_out [0.0, 0.7, 0.9099999999999999, 1]
final_alpha 1
q2 = 0.5700210291240112
CPU times: user 11.2 s, sys: 32.2 ms, total: 11.3 s
Wall time: 11.2 s


In [22]:
%%time



alphas = []
c = 0.3
b = 0.8
for i in range(30):
    alphas.append(1-((1-b)**i))


Als, A, qH_ls, qH_basic, alphas_out = hL.h_louvain_community_plus_last_step(alphas = alphas, 
                                                                            change_frequency = c)


print("alphas_out", alphas_out)
print("final_alpha", alphas_out[-1])
print("qH-basic =",qH_basic)
print("qH+last_step =",qH_ls)
    

alphas_out [0.0, 0.8, 0.96, 0.992, 1]
final_alpha 1
qH-basic = 0.5703178247693421
qH+last_step = 0.57389831072123
CPU times: user 1min 25s, sys: 136 ms, total: 1min 25s
Wall time: 1min 25s
